In [4]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import streaming_bulk
import glob
import ujson as json
from tqdm import tqdm
import os
import pandas as pd
from typing import Any, Dict, Iterable


In [108]:
with open('stopwords.txt', 'r') as f:
    stopwords_list = f.read().split("\n")


def create_fund_indx(es: Elasticsearch, doc_dims: int = 256):
    dense_paintings = {
        "settings": {
            "analysis": {
                "analyzer": {
                    "pl_analyzer": {
                        "type": "standard",
                        "filter": ["lowercase", "polish_stop", "polish_stem"],
                        "char_filter": ["html_strip"]
                    },
                },
                "filter": {
                    "polish_stop": {
                        "type": "stop",
                                "ignore_case": True,
                                "stopwords": ["_polish_"]  # plugin required
                    }
                }
            },
        },
        "mappings": {
            "properties": {
                # "text_vector": {
                #     "type": "dense_vector",
                #     "dims": doc_dims
                # },
                "description": {
                    "type": "text",
                    "analyzer": "polish"
                },
                "name": {
                    "type": "keyword"
                },
                "url": {
                    "type": "text"
                }
            }
        }
    }
    es.indices.delete("funds", ignore=[400, 404])
    es.indices.create(index="funds", body=dense_paintings)


def strip_extra_tags(txt):
    for tag in ('\n',
                r'<em>', r'</em>', "\t"
                ):
        txt = txt.replace(tag, "")
    txt = txt.replace(u'\xa0', u' ')
    return txt


def stream_fund_docs(src_df: pd.DataFrame) -> Iterable[Dict[str, Any]]:
    """Stream funds to the ES stack"""
    records = src_df.to_dict('records')
    for record in records:
        yield {
            "_index": "funds",
            "_source": {
                'description': strip_extra_tags(record['FullDesc']),
                'name': record['Name'],
                'URL': record['URL']
            }
        }


In [125]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

user = os.environ.get("USER")
password = os.environ.get("PASS")
addr = os.environ.get("ADDR")

src_df = pd.read_pickle('../pfr/PFR_free_txt.pkl')
es = Elasticsearch(
    hosts=[
        f"http://{user}:{password}@{addr}:9200"
    ]

)
create_fund_indx(es)
stream = stream_fund_docs(src_df=src_df)
for ok, response in streaming_bulk(es, actions=stream):
    if not ok:
        print(response)


/opt/homebrew/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [122]:
def simple_query(PKD: str):
    query = {
        'query': {
            'simple_query_string': {
                'query': PKD,
                "fields": ["description"]
            }
        },
        "highlight": {
            "fields": {
                "description": {}  # highlight here
            }
        }
    }

    results = es.search(index='funds', body=query)
    return results['hits']['hits']


def es_search(PKD: str):
    query = {
        'query': {
            "bool": {
                "must": {
                    "query_string": {
                        "query": PKD
                    }
                },
                # "should": [
                #     {"term": {"tags": "lat"}}
                # ],
                # "minimum_should_match": 1
            },
        },
        "highlight": {
            "fields": {
                "description": {}
            }
        }
    }
    results = es.search(index='funds', body=query)
    return results['hits']['hits']

simple_query("firma")

[{'_index': 'funds',
  '_type': '_doc',
  '_id': 'UD7cqH0BcKaLNnUZTEfb',
  '_score': 0.8329344,
  '_source': {'description': 'Przez Sukcesję z Przewodnikiem – Przez Sukcesję z PrzewodnikiemSkip to contentProjektSukcesjaDoradcyRegulaminFormularz kontaktowyBlogArtykułyWideo BlogO nasKontaktprojekt doradczy dla firm rodzinnychprojekt doradczy dla firm rodzinnychweź udział w projekcie, a nasi doradcy......przygotują diagnozępotrzeb wsparcia,by jak najlepiej dopasować usługi doradcze i szkoleniowe do Twoich potrzeb....pomogą Ci wypracować strategię sukcesji,by zaplanować proces sukcesji, z uwzględnieniem uwarunkowań Twojej firmy i rodziny....podpowiedzą jak rozpocząć działania,by realizując je konsekwentnie, przeprowadzićz powodzeniem proces sukcesji.O Projekcie“Przez Sukcesję z Przewodnikiem”W przygotowaniach do procesu sukcesji przedsiębiorcom rodzinnym przydatne może być wsparcie doradców, którzy – w ramach projektu dofinansowanego ze środków UE – zdiagnozują sytuację w firmie oraz wskaż